The solution requires a few Python libraries installed and environment variables to be configured. See explanation of each variables, its purpose and how to configure it properly.

In [ ]:
pip install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.9/386.9 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.2/325.2 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.8 MB/s eta 0:00:00


In [ ]:
import requests
import os
import openai
from google.colab import userdata

#define variables

#each case has a case ID, 1374 represents a case with about 40k "Enron" emails in Reveal's consulting envionment
case_id = 1374

#item id is used by Reveal to represent document internally, item 18557 represent Control number: DEMO-018557
itemid = 18557

#this user will be used to login, so make sure you have a valid login before getting started
username = "nexlp"

#Base URL is the root address of the Reveal website you want to work with
baseurl = "https://consulting.us-east-1.reveal11.cloud"

#defines what OpenAI models we will use to get score
model_name = "o1-mini"

#question will be used to build the prompt, you can modify the question to suit your needs.
question = '''Read the email below, provide a score from 1-10 to reprent level of 'Anger' found in the email,
              where 1 menas lowest and 10 represents highest. Also provide a quick explanation for the score. '''

#password and openai_key are set as environment variables
#password = os.environ.get('DEMO_PASSWORD')
#openai_key = os.environ.get('OPENAI')
password = userdata.get("DEMO_PASSWORD")
openai_key = userdata.get('OPENAI')

#the following variables will be populated by code
token = ""
userid = 0
body_text = ""

Once we have the variables configured, we can call Reveal's login API to retrieve authentication token and user id which will be used for later calls.

In [ ]:
#code to login to Reveal
url = baseurl + "/rest/api/v2/login"
headers = {
  "Content-Type": "application/json",
  "method": "POST",
  "user-agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML like Gecko) Chrome/110.0.0.0 Safari/537.36"
}

# Define login credentials (adjust with actual field names in LoginRequest schema)
login_payload = {
    "username": username,
    "password": password
}

try:
    # Send the login request
    response = requests.post(url, headers=headers, json=login_payload)

    #Check if the response was successful
    if response.status_code == 200:
        # Parse the token from the JSON response
        token = response.json().get("loginSessionId")  # Adjust if the token field has a different name in the response schema
        userid = response.json().get("userId")
        if token and userid:
            print("Login successful!")
        else:
            print("Token not found in response.")
    else:
        print(f"Failed to login. Status code: {response.status_code}. Response: {response.text}")

except requests.exceptions.RequestException as e:
    print("An error occurred:", e)

Login successful!


Once we successfully retrieved token and userid from login, we can then use them to make an API call and retrieve the email body of the document.

In [ ]:
# Request body matching the DocumentsRequest schema
documents_request = {
    "keyField": "ItemID",
    "combineDateTimeFields": True,
    "useFieldNames": False,
    "fieldProfileName": "Default",
    "documentIds": [itemid],
    "documentFields": ["Body Text"],
    "maxTextLength": 0
}

# Headers
headers = {
   "incontrolauthtoken": token,
   "Content-Type": "application/json",
   "method": "POST",
   "user-agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML like Gecko) Chrome/110.0.0.0 Safari/537.36"
}

# Make the POST request
try:
    response = requests.post(
        f"{baseurl}/rest/api/document",
        params={'caseId': case_id, 'userId': userid},
        json=documents_request,
        headers=headers
    )
    response.raise_for_status()  # Raise an error for bad status codes
except requests.exceptions.RequestException as e:
    print(f"An error occurred: {e}")
    exit()

# Process the response to extract the value of the first field from the first document
if response.status_code == 200:
    # Successful response
    documents = response.json()
    if documents:
        first_document = documents[0]
        fields = first_document.get('fields', [])
        if fields:
            first_field = fields[0]
            body_text = first_field.get('fieldValue')
            print(f"First field value: {body_text}")
        else:
            print("No fields found in the first document.")
    else:
        print("No documents returned in the response.")
elif response.status_code == 500:
    # Server error
    error_message = response.text
    print(f"Server Error: {error_message}")
else:
    # Unexpected status code
    print(f"Unexpected status code: {response.status_code}")
    print(response.text)


First field value: From: Matthew Lenhart
Sent: Monday, April 23, 2001 10:58:00 PM
To: Lisa Gillette
Subject: Re:

i didn't get thrown out of a casino.  i had to leave this bar b/c they over charged us by $300 so i argued w/ the manager and he said i had to leave.  i was pissed, but i didn't get physically removed or anything.  these guys at this club tried to rip us off and i was pissed.   i didn't out do my birthday either.  we drank a lot but it was over long periods of time.  it wasn't like on my birthday when we just slammed everything as fast as we could.  

***********
EDRM Enron Email Data Set has been produced in EML, PST and NSF format by ZL Technologies, Inc. This Data Set is licensed under a Creative Commons Attribution 3.0 United States License <http://creativecommons.org/licenses/by/3.0/us/> . To provide attribution, please cite to "ZL Technologies, Inc. (http://www.zlti.com)."
***********



With the body text in hand, we can now make a call to OpenAI, send the text and retrieve the results.

In [ ]:
#send to OpenAI and get "negative score" and reason
# Send the prompt to OpenAI's API
openai.api_key = os.getenv("OPENAIKEY", openai_key)

try:
    completion = openai.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {
                "role": "user",
                "content": question + " [START EMAIL] " + body_text + " [END EMAIL]"
            }
        ]
    )

    print(completion.choices[0].message)

except Exception as e:
    print(f"An error occurred: {e}")


ChatCompletionMessage(content='I would score the level of \'Anger\' in the email as a 4 out of 10. \n\nExplanation: The writer does express anger, specifically stating they were "pissed" twice in relation to being overcharged and the situation that ensued. However, the tone of the email is not overtly aggressive or hostile; it\'s more explanatory and somewhat defensive about the situation. The writer seems upset but does not resort to strong language or personal attacks, which indicates a moderate, rather than high, level of anger.', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None)


Here is the original email body: "i didn't get thrown out of a casino.  i had to leave this bar b/c they over charged us by $300 so i argued w/ the manager and he said i had to leave.  i was pissed, but i didn't get physically removed or anything.  these guys at this club tried to rip us off and i was pissed.   i didn't out do my birthday either.  we drank a lot but it was over long periods of time.  it wasn't like on my birthday when we just slammed everything as fast as we could. "

And here is the expected response from AI:

I would score the level of 'Anger' in the email as a 4 out of 10.

Explanation: The writer does express anger, specifically stating they were "pissed" twice in relation to being overcharged and the situation that ensued. However, the tone of the email is not overtly aggressive or hostile; it\'s more explanatory and somewhat defensive about the situation. The writer seems upset but does not resort to strong language or personal attacks, which indicates a moderate, rather than high, level of anger
